## Call WOfS using bounding boxes
This notebook merges gauge data with satellite data for multiple reservoirs at a time (in the 00_Library diresctory). It also makes the depth to surface area relatiosnhips for the reservoirs. It's my main notebook for this project and closest to what I have as the final product. Bounding boxes are used to get the exact extents of reservoirs for the depth-to-surface area relationship. It's an automated way to call the right satellite data accurately.

In [2]:
import os
import xarray as xr
import numpy as np
import pandas as pd
import csv
import rasterio.crs
from tqdm.auto import tqdm #this one is a loading bar, it's cool to add loading bars to loops
from pandas import DataFrame
import geopandas as gpd
import matplotlib.gridspec as gs
import matplotlib.pyplot as plt
from matplotlib import pyplot
import datacube

import sys
sys.path.append('../../Scripts')
from dea_spatialtools import xr_rasterize
from datacube.utils import geometry 
from datacube.utils.geometry import CRS
from datacube.utils import masking
from datacube.helpers import ga_pq_fuser, write_geotiff
#from digitalearthau.utils import wofs_fuser
#import DEAPlotting, DEADataHandling
import warnings
warnings.filterwarnings('ignore', module='datacube')
%load_ext autoreload
%autoreload 2

/env/lib/python3.6/site-packages/cligj/__init__.py:17: FutureWarning: cligj 1.0.0 will require Python >= 3.7
  warn("cligj 1.0.0 will require Python >= 3.7", FutureWarning)


## Dask load the satellite data for all the reservoirs
I made a shapefile in ArcMAP that has bounding boxes of the reservoirs identified in 00_Library_reservoirs. This is what we will use for the query, so wofs knows where to make the picture for each reservoir. The following code blocks were copied from a DEA notebook called 'Open and run analysis on multiple polygons'. In this case the multiple polygons are my bounding boxes from the geodataframe above. First you make a query with no x, y points and no CRS. Just the time. Then you loop the location for the query using a datacube package called geomoetry. Put the dc.load() line in the loop (I'm going to dask load, not load actual images, I'll do that later after I've merged with the gauges). 

In [3]:
gdf = gpd.read_file('00_Lib_bound/00_Lib_bound.shp')

query = {'time': ('01-01-1988', '09-12-2020')} 
         #'crs': 'EPSG:3577'}
dc = datacube.Datacube(app='dc-WOfS')

results = {} 

#tqdm is gonna make the bar. tqdm is Arabic abbreviation for 'progress'
for index, row in tqdm(gdf.iterrows(), total=len(gdf)):
    geom = geometry.Geometry(geom=row.geometry, crs=gdf.crs)
    query.update({'geopolygon': geom})
    
    wofs_albers= dc.load(product = 'wofs_albers', dask_chunks = {}, group_by='solar_day', **query)
    
    poly_mask = xr_rasterize(gdf.iloc[[index]], wofs_albers)
    wofs_albers = wofs_albers.where(poly_mask, other=wofs_albers.water.nodata) #put other or all the data turns into 0
    
    results.update({str(row['gauge_ID']): wofs_albers}) #The handle for dictionary objects is the gauge ID

## loop read all the csv files in 00_Library
Now we have a library of the wofs data with the gauge ID as the key. We now need a library of the depth data with, again, the gauge ID as the key. Then we can match them up later. 

In [ ]:
#make a list of the file names so we can call them with pandas
file_list = []

directory = '00_Library'
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        file_list.append(os.path.join(directory, filename))

#Read the gauge files twice, once to get ID and second to get the data. Append them together in a dictionary
#May as well make a list of IDs here because we will probably use it later
data_dict = {}        
ID_list = []
#let's use tqdm again to make a progress bar. The bar is so cool I love this module
#I'm gonna use tqdm on literally all of my loops now
for i in tqdm(file_list, total=len(file_list)):
    df = pd.read_csv(i, nrows=1, escapechar='#')
    column = df.iloc[:,[1]] #This is the column with the ID in it
    ID = list(column)
    ID = ID[0]
    ID = df.at[0, ID]
    ID_list.append(str(ID))
    
    data = pd.read_csv(i, error_bad_lines = False, skiprows=9, escapechar='#',
                         parse_dates=['Timestamp'], 
                         index_col=('Timestamp'),
                        date_parser=lambda x: pd.to_datetime(x.rsplit('+', 1)[0]))
    data = data.drop(columns=['Quality Code', 'Interpolation Type'])
    data_dict.update({str(ID): data})

## Make a function that generates the depth slices of a reservoir and gets the depth to surface area relationship
This is my first time writing a function, Matthew and Bex from DEA helped me write it. 

In [ ]:
def image_prod(ID_caller, gauge_data, wofs_albers, make_plots = False) -> 'depth slices': 
    """
    This function takes the gauge data and the wofs data,
    cloud masks the images and counts the pixels in each depth slice.
    It returns a list of all the surface areas per depth.
    
    """
    #Get the depth range and intervals
    gauge_data = gauge_data.dropna()
    depth_integers = gauge_data.astype(np.int64)
    max_depth = depth_integers.Value.max()
    min_depth = depth_integers.Value.min()
    integer_array = depth_integers.Value.unique()
    integer_list = integer_array.tolist()
    
    gauge_data_xr = gauge_data.to_xarray() #convert gauge data to xarray
    merged_data = gauge_data_xr.interp(Timestamp=wofs_albers.time) #use xarrays .interp() function to merge

    surface_area_list = []

    for i in tqdm(integer_list, leave = False):
        specified_level = merged_data.where((merged_data.Value > i) & 
                                        (merged_data.Value < i+1), drop=True)
        date_list = specified_level.time.values[:150] #caps images at x per slice (way faster)
        n_images_used = int(len(date_list))
        specified_passes = wofs_albers.sel(time=date_list).compute() #This .compute() Xarray function loads actual images
        #cloudmask (Claire Krause wrote this for me)
        #print(specified_passes.water)
        cc = masking.make_mask(specified_passes.water, cloud=True)
        ncloud_pixels = cc.sum(dim=['x', 'y'])
        # Calculate the total number of pixels per timestep
        npixels_per_slice = (specified_passes.water.shape[1] * 
                             specified_passes.water.shape[2])
        cloud_pixels_fraction = (ncloud_pixels / npixels_per_slice)
        clear_specified_passes = specified_passes.water.isel(
            time=cloud_pixels_fraction < 0.2)
        wet = masking.make_mask(clear_specified_passes, wet=True).sum(dim='time')
        dry = masking.make_mask(clear_specified_passes, dry=True).sum(dim='time')
        clear = wet + dry
        frequency = wet / clear
        frequency = frequency.fillna(0)  

        #Get area from the satellite data
        #get the frequency array
        frequency_array = frequency.values
        #Turn any pixel in the frequency array with a value greater than 0.2 into a pixel of value 1
        #if the pixel value is 0.2 or lower it gets value 0
        is_water = np.where((frequency_array > 0.2),1,0)
        #give the 'frequency' xarray back its new values of zero and one
        frequency.values = is_water
        #sum up the pixels
        number_water_pixels = frequency.sum(dim=['x', 'y'])
        #get the number
        number_water_pixels = number_water_pixels.values.tolist()
        #multiply by pixel size to get area in m2
        area_m2 = number_water_pixels*(25*25)

        surface_area_list.append([ID_caller, i, area_m2, n_images_used])
        #print('This is the area as calculated from wet pixels at', i, 'meters', area_m2)

        #Plotting the image
        if make_plots:
            frequency.plot(figsize = (7,5))
        
    return surface_area_list

## Run the function for all of the reservoirs

In [ ]:
array_list = []
for ID in tqdm(ID_list, total=len(ID_list)):
    if (ID in data_dict.keys()) and (ID in results.keys()):
        data = image_prod(ID, data_dict[ID], results[ID], make_plots = False)
        array_list.append(data)
        del data
    else:
        print('we didnt find', ID)        

## Make a look up table of depth to surface area for each gauge ID

In [ ]:
look_up_table = []
for i in array_list:
    df = DataFrame(i, columns = ['ID', 'Depth', 'Surface Area', 'number of images'])
    look_up_table.append(df)

df = pd.concat(look_up_table)
df    

## save the look up table as a csv file

In [ ]:
df.to_csv('depth_to_surface_tables.csv')